---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [20]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import re

    with open('university_towns.txt', 'r') as unitowntext:
        statetowndict = {}
        currentstate = ""
        for line in unitowntext:
            if('[ed' in line):
                currentstate = line.split('[ed')[0]
                statetowndict[currentstate] = []
            else:
                statetowndict[currentstate].append(line)

    stmatrix = []
    for key, val in statetowndict.items():
        state = key
        for town in val:
            stmatrix.append([state,town])

    unitownsdf = pd.DataFrame(stmatrix, columns=['State','RegionName'])

    def cleantregionname(x):
        return x.split(' (')[0].strip()

    unitownsdf['RegionName'] = unitownsdf['RegionName'].apply(cleantregionname)
    
    return unitownsdf

In [27]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    '''
    gdpdf = pd.read_excel('gdplev.xls',
                     skiprows = 7,
                     usecols = [4, 6],
                     names = ['Quarter', 'GDP'])

    index_2000_q1 = gdpdf[gdpdf['Quarter'] == '2001q1'].index.values[0]

    recession_started = False
    recession_start = ""
    for i in gdpdf.index[index_2000_q1 + 2: ]:
        gdp = gdpdf.loc[i, 'GDP']
        gdp_1 = gdpdf.loc[i-1, 'GDP']
        gdp_2 = gdpdf.loc[i-2, 'GDP']
        if((gdp < gdp_1) & (gdp_1 < gdp_2) & (recession_started == False)):
            recession_started = True
            recession_start = gdpdf.loc[i-1, 'Quarter']
    
    return recession_start

In [28]:
get_recession_start()

'2008q3'

In [43]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdpdf = pd.read_excel('gdplev.xls',
                     skiprows = 7,
                     usecols = [4, 6],
                     names = ['Quarter', 'GDP'])

    recession_start_i = gdpdf[gdpdf['Quarter'] == get_recession_start()].index.values[0]

    for i in gdpdf.index[recession_start_i + 2: ]:        
        gdp = gdpdf.loc[i, 'GDP']
        gdp_1 = gdpdf.loc[i-1, 'GDP']
        gdp_2 = gdpdf.loc[i-2, 'GDP']
        if((gdp > gdp_1) & (gdp_1 > gdp_2)):
            recession_end = gdpdf.loc[i, 'Quarter']
            break
            
    return recession_end

In [44]:
get_recession_end()

'2009q4'

In [30]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdpdf = pd.read_excel('gdplev.xls',
                     skiprows = 7,
                     usecols = [4, 6],
                     names = ['Quarter', 'GDP'])

    recession_start_i = gdpdf[gdpdf['Quarter'] == get_recession_start()].index.values[0]
    recession_end_i = gdpdf[gdpdf['Quarter'] == get_recession_end()].index.values[0]
    return gdpdf.loc[gdpdf['GDP'][recession_start_i:recession_end_i].idxmin(), ['Quarter']][0]

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housesdf = pd.read_csv('City_Zhvi_AllHomes.csv')
    housesdf['State'].replace(states, inplace = True)
    housesdf = housesdf.set_index(['State', 'RegionName']).drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis = 1)

    # Drop all columns before 2000
    cols = [c for c in housesdf.columns if c.lower()[:3] == '199']
    housesdf.drop(cols, axis = 1, inplace = True)

    # Convert columns to datetimes
    housesdf.columns = pd.to_datetime(housesdf.columns)
    # Resample dates based on quarters and take the average
    housesdf = housesdf.resample('Q', axis = 1).mean()
    # Convert column names to string quarters
    housesdf = housesdf.rename(columns = lambda x: str(x.to_period('Q')).lower())

    return housesdf

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Get quarter before recession start
    housesdf = convert_housing_data_to_quarters()
    recession_start_index = housesdf.columns.get_loc(get_recession_start())
    q_before_recession = housesdf.columns[recession_start_index - 1]
    recession_bottom = get_recession_bottom()
    
    # Get uni town housing data between pre-recession start, and recession bottom
    uni_houses = housesdf.loc[:, q_before_recession:recession_bottom].iloc[get_list_of_university_towns().index,:].dropna().sample(n=500)
    reg_houses = housesdf.loc[:, q_before_recession:recession_bottom][~housesdf.index.isin(uni_houses.index)].dropna().sample(n=500)
    
    # Add column for ratio between house price pre-recession and recession bottom
    uni_houses['ratio'] = uni_houses[q_before_recession].div(uni_houses[recession_bottom])
    reg_houses['ratio'] = reg_houses[q_before_recession].div(reg_houses[recession_bottom])
    
    t, p = ttest_ind(uni_houses['ratio'], reg_houses['ratio'], nan_policy = 'raise')
    return (True, p, 'university town')